In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.keras_models as Models
import core
import keras as keras


MODEL_DIR = f"models/assignment_transformer_HLF"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    load_config["nominal"], max_events=4_000_000
)
X_train, y_train, _, _ = DataProcessor.split_even_odd()
del DataProcessor

2025-12-17 15:48:39.499157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765982919.521906 3705526 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765982919.529480 3705526 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765982919.547794 3705526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765982919.547815 3705526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765982919.547818 3705526 computation_placer.cc:177] computation placer alr

In [2]:
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name=r"Transformer")

TransformerMatcher.build_model(
    hidden_dim=32,
    num_layers=4,
    num_heads=8,
    dropout_rate=0.1,
    compute_HLF=True,
    log_variables=True,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)
TransformerMatcher.model.summary()

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1765983023.459692 3705526 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.
Submodel inputs:  [<KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs>, <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs>]
Input tensor:  <KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs> jet_inputs
Input tensor:  <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs> lep_inputs
Input tensor:  <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs> met_inputs


I0000 00:00:1765983025.620502 3707286 service.cc:152] XLA service 0x7f79400017b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765983025.620526 3707286 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-17 15:50:25.646386: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1765983025.705837 3707286 cuda_dnn.cc:529] Loaded cuDNN version 91500


150/943 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

...

Model: "FeatureConcatTransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 7)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 6)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ compute_high_level… │ (None, 6, 2, 2)   │          0 │ jet_input_transf… │
│ (ComputeHighLevelF… │                   │            │ jet_mask[0][0],   │
│                     │                   │            │ lep_input_transf… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hlf_input_normaliz… │ (None, 6, 2, 2)   │          9 │ compute_high_lev… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 6)      │         13 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 7)      │         15 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 4)      │          0 │ hlf_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 12)        │          0 │ lep_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 11)     │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 37,165 (145.19 KB)

 Trainable params: 37,123 (145.01 KB)

 Non-trainable params: 42 (184.00 B)

In [3]:
TransformerMatcher.train_model(
    epochs=50,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=8,
            restore_best_weights=True,
            verbose=1,
        ),
    ],
)

Epoch 1/50


2025-12-17 15:50:46.835888: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1557/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1884 - loss: 0.1465

2025-12-17 15:51:11.196990: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1890 - loss: 0.1465

2025-12-17 15:51:22.437681: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-17 15:51:24.813084: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 50s 18ms/step - accuracy: 0.1891 - loss: 0.1465 - val_accuracy: 0.6680 - val_loss: 0.0822 - learning_rate: 1.0000e-04
Epoch 2/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6520 - loss: 0.0876 - val_accuracy: 0.7228 - val_loss: 0.0710 - learning_rate: 1.0000e-04
Epoch 3/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6949 - loss: 0.0774 - val_accuracy: 0.7389 - val_loss: 0.0682 - learning_rate: 1.0000e-04
Epoch 4/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.7156 - loss: 0.0734 - val_accuracy: 0.7532 - val_loss: 0.0664 - learning_rate: 1.0000e-04
Epoch 5/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.7279 - loss: 0.0712 - val_accuracy: 0.7620 - val_loss: 0.0653 - learning_rate: 1.0000e-04
Epoch 6/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.7394 - loss: 0.0697 - val_accuracy: 0.7687 - val_loss: 0.0643 - learning_rate: 1.0000e-04
Epoch 7/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step -

KeyboardInterrupt: 

In [ ]:
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")